# Summary

Loop through prerequisites, and establish a dictionary where the keys are the first element of each sub-list. And the prerequisites of each course will be the corresponding value which is a list of prerequisite courses.

Then we iterate the keys one by one. At each key, we loop through its prereq list. Then for each prereq, we also check its prereqs, and if all its prereqs have been taken, or that it doesn't exist in the requirement hashmap, then mark the original key as take

check whether it's been taken. If not, take it (visited) and decrement the numCourses variable by 1.

We break out if numCourses has reached 0.

At the end if we can successfully loop through everything, then we can return True.


## Time Complexity
* Building of the hashmap will take $O(E)$ where E is the number of prerequisites (length of the prereqs list)
* Then `check_eligibility` will be called maximally by the number of courses that exist, $O(V)$
* Each `check_eligibility` can at most then need to loop through all of prereqs again, because each prereq may have its own prereqs 

So the last two bullet points together will take $O(V+E)$

## Space Complexity
$O(V + E)$ because `taken`, and `processing` can each be at most $O(V)$. And for `hashmap` it will be $O(E)$ for each prereq



In [20]:
from typing import List


class SolutionDebug:
    def canFinish(self, numCourses: int, prerequisites: List[List[int]]) -> bool:
        hashmap = dict()
        taken = set()
        processing = set()

        for prereq_list in prerequisites:
            if prereq_list[0] not in hashmap:
                hashmap[prereq_list[0]] = prereq_list[1:]
            else:
                hashmap[prereq_list[0]].extend(prereq_list[1:])
        
        print(f"Hashmap = {hashmap}")

        def check_eligibility(prereqs, numCourses):
            for prereq in prereqs:
                if numCourses <= 0:
                    return False, numCourses
                if prereq in taken:
                    continue
                elif prereq in processing:
                    print(f"{prereq} already in processing={processing}")
                    return False, -1
                elif prereq not in hashmap:
                    taken.add(prereq)
                    numCourses -= 1
                elif prereq in hashmap:
                    processing.add(prereq)
                    print(f"Processing = {processing} to check eligibility")
                    eligibility, numCourses = check_eligibility(hashmap[prereq], numCourses)
                    if not eligibility:
                        return False, numCourses
                    processing.remove(prereq)
            return True, numCourses
        
        for course, prereqs in hashmap.items():
            processing.add(course)
            print(f"Processing = {processing} to check eligibility")
            eligibility, numCourses = check_eligibility(prereqs, numCourses) 
            if eligibility:
                taken.add(course)
                numCourses -= 1
            else:
                return False
            processing.remove(course)
        return True
            
                

In [21]:
prerequisites = [[1,4],[2,4],[3,1],[3,2]]
numCourses = 5

s = SolutionDebug()
s.canFinish(numCourses, prerequisites)

Hashmap = {1: [4], 2: [4], 3: [1, 2]}
Processing = {1} to check eligibility
Processing = {2} to check eligibility
Processing = {3} to check eligibility


True